In [1]:
import sys
print(sys.version)

3.7.12 (default, Oct 13 2021, 06:53:03) 
[Clang 13.0.0 (clang-1300.0.29.3)]


In [2]:
%load_ext autoreload
%autoreload 0

import os, json
from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from spread import get_spread_data, get_current_rankings
from collections import defaultdict

# https://www.pro-football-reference.com/years/2021/fantasy.htm

SALARY_FILE = './data/12_29.csv'
WEEK = 17
MIN_SALARY = 4699
WEIGHTED = True

ACTIVE_FILE = './data/active.csv'
MAX_PLAYED = WEEK - 1
print('ready')

ready


In [3]:
ranking_df = get_current_rankings(WEEK, 'MTI1MjcxNDMyNA==')

def get_abbr(x):
    try:
        if type(x) is not dict:
            x = eval(x)
        return x['team_abbr']
    except Exception as e:
        print(e,x)
        

print(ranking_df.shape)
ranking_df['team'] = ranking_df['nav'].apply(get_abbr)
num_teams = ranking_df.shape[0]


return cached data week 17
(32, 49)


In [4]:
rankings = {x['team']: x for x in ranking_df.to_dict('records')}
# rankings
teams = set(rankings.keys())
print(teams, len(teams))
rankings['LV'] = rankings['OAK']
rankings['JAC'] = rankings['JAX']
del rankings['OAK']
# rankings['WAS']

{'LAC', 'LAR', 'JAX', 'DAL', 'CHI', 'PHI', 'HOU', 'SEA', 'TB', 'MIN', 'DET', 'IND', 'BAL', 'DEN', 'KC', 'CAR', 'PIT', 'GB', 'TEN', 'ARI', 'CIN', 'NE', 'NO', 'WAS', 'OAK', 'MIA', 'SF', 'NYG', 'NYJ', 'BUF', 'ATL', 'CLE'} 32


In [5]:

spread_df = get_spread_data(WEEK)
favor_map = {}
# https://madduxsports.com/how-to-read-nfl-odds-lines.html"
for index, row in spread_df.iterrows():
    favor_map[row['HomeTeam']] = row['PointSpread']
    favor_map[row['AwayTeam']] = -row['PointSpread']
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']



return cached data week 17


In [6]:
df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 2
half_teams = int(len(ordered_teams)/2)
top_teams = ordered_teams#[team_offset:team_offset+int(half_teams)]
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = [*questionable_players, *low_salary_players]
excluded_teams = list(df[(df['Position'] == 'D') & df['Team'].isin(excluded_teams)]['Name'])

if excluded_teams:
    excluded_players.extend(excluded_teams)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")
custom_adds = ['Keenan Allen', 'Joe Mixon', 'Justin Jackson', 'Dawson Knox', "George Kittle", 'Mark Andrews', 'Stefon Diggs', 'Ezekiel Elliott', 'Tyreek Hill', 'Deshaun Watson', 'Justin Herbert', 'Hunter Henry']

if excluded_players:
    excluded_players.extend(custom_adds)
    
questionable_df = df[(df['Name'].isin(set(questionable_players)))]
    
excluded_df = df[(df['Name'].isin(set(excluded_players)))]
    
df = df[(~df['Name'].isin(set(excluded_players)))]# & (df['FPPG'] > 0) & (df['Played'] > 0)

             FPPG      Played        Salary  Tier  Unnamed: 14  Unnamed: 15
count  923.000000  923.000000    923.000000   0.0          0.0          0.0
mean     4.182835    6.790899   4963.271939   NaN          NaN          NaN
std      4.613859    4.844954    989.327495   NaN          NaN          NaN
min     -0.666667    1.000000   3000.000000   NaN          NaN          NaN
25%      0.741667    2.000000   4500.000000   NaN          NaN          NaN
50%      2.841667    6.000000   4500.000000   NaN          NaN          NaN
75%      5.889835   11.000000   5300.000000   NaN          NaN          NaN
max     24.654667   18.000000  10200.000000   NaN          NaN          NaN
ordered_teams [('NE', -15.5), ('BUF', -14.5), ('SF', -13.5), ('TB', -13.0), ('IND', -7.5), ('NO', -7.0), ('SEA', -7.0), ('GB', -6.5), ('CHI', -6.0), ('KC', -5.5), ('LAC', -5.5), ('DAL', -5.5), ('TEN', -3.5), ('PHI', -3.5), ('LAR', -3.5), ('CLE', -2.5), ('PIT', 2.5), ('MIA', 3.5), ('WAS', 3.5), ('BAL', 3.5), ('CIN',

/usr/local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)


In [7]:
print(winning_teams)

['BUF', 'CHI', 'KC', 'TEN', 'IND', 'NE', 'TB', 'PHI', 'LAR', 'LAC', 'SF', 'DAL', 'NO', 'SEA', 'GB', 'CLE']


In [8]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-3 # take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1)]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names if os.path.isfile(f)]
print(f"Using {len(history_dfs)} weeks of history")
historic_data=pd.concat(history_dfs)

if len(historic_data) > 0:
    REPLACE_MAP = {
        'LA': 'Los Angeles',
        '.':'',
    }

    def name_map(x):
        result = ' '.join(x.split(', ')[::-1])
        for k in REPLACE_MAP:
            result = result.replace(k, REPLACE_MAP[k])
        return result


    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)
    
    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
print(len(historic_averages))

['./history/week14.csv', './history/week15.csv', './history/week16.csv', './history/week17.csv'] 4
Using 3 weeks of history
609


In [9]:
excluded_bonus = defaultdict(lambda: 0)

for index, p in questionable_df.iterrows():
    if p['Position'] in ['TE', 'WR', 'RB']:
        points = p['FPPG']
        if points > 5:
            amt = min(points * .1, 1)
            print('bonus', p['Name'], amt)
            excluded_bonus[p['Team']] += amt
        
print(excluded_bonus)

bonus Travis Kelce 1
bonus Eli Mitchell 1
bonus D'Andre Swift 1
bonus Mike Evans 1
bonus Darrell Henderson Jr. 1
bonus Calvin Ridley 1
bonus Leonard Fournette 1
bonus James Conner 1
bonus Mike Williams 1
bonus Darren Waller 1
bonus Brandin Cooks 1
bonus Miles Sanders 0.8683333079020183
bonus Clyde Edwards-Helaire 1
bonus Elijah Moore 1
bonus Jordan Howard 0.9816666920979819
bonus T.J. Hockenson 0.9566666920979818
bonus Rhamondre Stevenson 0.8109999847412109
bonus Jamison Crowder 0.74636362249201
bonus Cole Beasley 0.7714285714285714
bonus Julio Jones 0.5622222052680121
bonus J.D. McKissic 0.9672727411443538
bonus Tre'Quan Smith 0.6177777608235678
bonus Gabriel Davis 0.75
bonus Nick Westbrook-Ikhine 0.5600000234750601
bonus Jalen Guyton 0.5100000108991352
bonus Bryan Edwards 0.5985714503696988
bonus Laviska Shenault Jr. 0.6035714285714286
bonus Zach Pascal 0.5214285714285715
bonus Nelson Agholor 0.6315384498009315
bonus Josh Reynolds 0.7166666666666668
bonus David Johnson 0.501666673024

In [10]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./data/active.csv, (293, 19)


In [11]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2, 3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]               

In [12]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
    
m_score

{'D': 6.504761904761905,
 'QB': 6.454139496106781,
 'RB': 6.993283036412837,
 'TE': 6.928255229099322,
 'WR': 6.924882303639204,
 'DEF': 6.504761904761905}

In [13]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)-1
max_salary = 8500
print(max_salary)

8500


In [14]:
df[df['Position'] == 'D'].shape

(28, 19)

In [15]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - 100
ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 59900, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}


In [16]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

In [33]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 12
AVERAGE_WEIGHT = 0.3

defenses = []

historic_data_used = 0

for p in players:
    p.average_score = m_score[p.pos]
    name = p.name.replace('.', '')
    
    if 'Bur' in name or 'Dav' in name:# or p.pos == 'TE':
        print(p)
        
    if WEIGHTED:
        history_key = name_map(p.name) if p.pos != 'D' else p.team.lower()
        history_value = historic_averages.get(history_key)
        if history_value:
            new_score = AVERAGE_WEIGHT*p.proj + (1-AVERAGE_WEIGHT)*history_value
            p.proj = new_score
            historic_data_used+=1
        else:
            new_score = None
            #print('no historic data', history_key)
            
        
        rank_bonus = -favor_map[p.team]/FAVOR_DIVISION
        teams = p.matchup.split('@')

        is_home = p.team == teams[1]
        if is_home:
            rank_bonus += 0.22

        teams.remove(p.team) # remove current players team
        opponent = teams[0]

        if p.pos == 'D':
            current_def = rankings[p.team]['defensive_rating_rank']
            opp_off = rankings[opponent]['offensive_rating_rank']
            overall_diff = (opp_off - current_def)
            # print(p.team, overall_diff, opp_off, current_def)
            opp_bonus = excluded_bonus[opponent]
            rank_bonus += opp_bonus
            
        else:
            if p.pos == 'QB':
                # negative (downside for QB injuries)
                injury_bonus = -excluded_bonus[p.team]
            else:
                injury_bonus = excluded_bonus[p.team] 
                
            rank_bonus += injury_bonus
            overall_diff = (rankings[opponent]['overall_rating_rank'] - rankings[p.team]['overall_rating_rank']) # should be between.

        rank_bonus += overall_diff/(num_teams-1)*1.5
        p.proj = p.proj + rank_bonus
        
        if p.pos == 'D':
            defenses.append((p.team, opp_bonus, rank_bonus, p.proj))
        elif p.pos == 'RB':
            pass
            #print(history_key, p.proj, history_value)
        
#             opp_defense = rankings[opponent]['defensive_rating_rank']
#             defense_bonus = (opp_defense-(num_teams/2))/(num_teams/3)
#             if False:
#                 print(p.team, opponent, opp_defense, rank_bonus, overall_diff, defense_bonus, rank_bonus)
#             p.proj += defense_bonus

print(f"players {len(players)}")
print("historic data used", historic_data_used, len(players)) 
# print(players)

# for x in sorted(defenses, key=lambda x: x[-1], reverse=True):
#     print(x)
# print(excluded_bonus)

[QB] Joe Burrow           CIN KC@CIN ($7,700, 19.55999959309896 (10.82)), 
[RB] David Montgomery     CHI NYG@CHI ($7,200, 13.272727272727273 (6.28)), 
[QB] Davis Mills          HOU HOU@SF ($6,400, 11.80000027743253 (3.06)), 
[QB] Davis Webb           BUF ATL@BUF ($6,000, -0.30000001192092896 (-9.04)), 
[QB] David Blough         DET DET@SEA ($6,000, 0.6000000238418579 (-8.14)), 
[RB] Rex Burkhead         HOU HOU@SF ($5,600, 6.0666669209798165 (-0.93)), 
[RB] Mike Davis           ATL ATL@BUF ($5,400, 6.913333129882813 (-0.08)), 
players 293
historic data used 217 293


In [40]:
player_settings = PlayerPoolSettings()
LOCKED = []
BLOCKED_TEAMS = []

# LOCKED = ['Justin Herbert','Ronald Jones II','Dallas Goedert','Miami Dolphins']# 'Miami Dolphins']#['Justin Herbert',  'Tee Higgins', 'Ronald Jones II', 'David Montgomery', 'Miami Dolphins']
# LOCKED = [ 'Justin Herbert', 'Ronald Jones II','Dallas Goedert', 'Cincinnati Bengals', 'David Montgomery']
# LOCKED = ['Joe Burrow', 'Brandon Aiyuk']
# LOCKED = ['Justin Jackson', 'Darrel Williams', 'Dak Prescott', 'Dalton Schultz']
# LOCKED = ['Darrel Williams']
# LOCKED = ['Joe Burrow']
# LOCKED = ['Tee Higgins']
# LOCKED = ['Dallas Goedert', 'Byron Pringle']
# LOCKED = ['David Montgomery', 'Byron Pringle']
LOCKED = ['Dallas Goedert', 'Dak Prescott']

BLOCKED_TEAMS = ['SEA', 'LV']#, 'ARI', 'LV']

constraints = LineupConstraints(locked=LOCKED)

MIN_PROJ = 8
min_favored = -6
MIN_LIMIT = -8

roster = None
best_roster = None
best_score = 0

get_score = lambda roster: sum([p.proj for p in roster.players])

while min_favored >= MIN_LIMIT:
    def block_function(p):
        store = p.kv_store
        played = int(float(store.get('Played', -1)))
        
        if 'Ronald' in p.name or 'Davi' in p.name:
            return False

        should_skip = ((p.team in BLOCKED_TEAMS or p.proj < MIN_PROJ) and p.pos != 'D') or (p.cost > max_salary) or (played >= 0 and played < MAX_PLAYED-4) or played > MAX_PLAYED + 1 or (p.pos == 'QB' and p.proj < 18) or (favor_map[p.team] > -min_favored) 
    
        #print(p.name, played, MAX_PLAYED)
        return should_skip

    opt_settings = OptimizerSettings(
        custom_rules=[
            CustomRule(
                group_a=lambda p: p,
                group_b=block_function, 
                comparison=lambda sum, a, b: sum(b) == 0
            ),
        ],
        min_teams=7
    )

    roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=False,
        optimizer_settings=opt_settings,
        constraints=constraints,
        player_settings=player_settings
    )
    
    if roster:
        # total_salary = sum([p.cost for p in roster.players])
        print(roster)
        print('min_favored', min_favored)# negative means players' teams can lose
        print(f"spread weighted: {WEIGHTED}\n\n")
        current_score = get_score(roster)
        if not best_score or current_score > best_score:
            best_score = current_score
            best_roster = roster

    min_favored -= 1
    
    if min_favored >= MIN_LIMIT:
        roster = None
        
if not roster:
    print("No solution")

+----------+----------------+------+---------+--------+--------------------+----------+--------+
| Position | Player         | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+----------------+------+---------+--------+--------------------+----------+--------+
| QB       | Dak Prescott   | DAL  | ARI@DAL |  7,400 | 19.144619073776973 |    10.41 | LOCK   |
| RB       | Damien Harris  | NE   | JAC@NE  |  7,400 | 27.688522885356488 |    20.70 |        |
| RB       | Sony Michel    | LAR  | LAR@BAL |  7,100 |  15.05016129032258 |     8.06 |        |
| WR       | Ja'Marr Chase  | CIN  | KC@CIN  |  7,600 | 12.096763318789902 |     5.17 |        |
| WR       | Jaylen Waddle  | MIA  | MIA@TEN |  7,000 | 12.356719994142127 |     5.43 |        |
| WR       | Tee Higgins    | CIN  | KC@CIN  |  7,000 | 15.413378684625236 |     8.49 |        |
| WR       | Christian Kirk | ARI  | ARI@DAL |  6,400 | 14.789161841982885 |     7.86 |        |
| TE       | Dallas Goedert | 

In [19]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
roster = best_roster
sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
net_score = 0
for p in sorted_players:
    advantage = favor_map[p.team]
    name = p.name.replace('.', '')
    print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj} {p.kv_store['Played']} {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
    net_score += advantage
print(f"---\nTotal adv: {net_score}\n")
roster.players[-1].__dict__



Damien Harris                -15.5                28.3 27.688522885356488 13.0 2.2825384599734577
Devin Singletary             -14.5  14.766666666666666 17.962944822531142 15.0 2.5214285714285714
Patrick Mahomes               -5.5  24.413333333333338 23.21257001454343 15.0 2
Dalton Schultz                -5.5  11.933333333333332 11.94766672770182 15.0 0
Dallas Cowboys                -5.5                     21.154847900109356 15.0 0
Sony Michel                   -3.5  13.466666666666667 15.05016129032258 15.0 2.9266666666666667
Jaylen Waddle                  3.5                     12.356719994142127 14.0 0
Tee Higgins                    5.5                18.2 15.413378684625236 13.0 0
Amon-Ra St Brown               7.0  17.433333333333334 17.455849583709295 14.0 3.5113333892822265
---
Total adv: -34.0



{'pos': 'D',
 'name': 'Dallas Cowboys',
 'cost': 4200.0,
 'team': 'DAL',
 'matchup': 'ARI@DAL',
 'proj': 21.154847900109356,
 'average_score': 6.504761904761905,
 'projected_ownership_pct': 0,
 'lineup_count': 0,
 'marked': None,
 'lock': False,
 'position_lock': False,
 'ban': False,
 'position_ban': False,
 'multi_position': False,
 'possible_positions': 'D',
 'kv_store': OrderedDict([('', '766'),
              ('Id', '69306-12530'),
              ('Position', 'D'),
              ('First Name', 'Dallas'),
              ('Nickname', 'Dallas Cowboys'),
              ('Last Name', 'Cowboys'),
              ('FPPG', '11.933333333333335'),
              ('Played', '15.0'),
              ('Salary', '4200'),
              ('Game', 'ARI@DAL'),
              ('Team', 'DAL'),
              ('Opponent', 'ARI'),
              ('Injury Indicator', ''),
              ('Injury Details', ''),
              ('Tier', ''),
              ('Unnamed: 14', ''),
              ('Unnamed: 15', ''),
          

In [20]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])

KC Patrick Mahomes 23.21257001454343 6.454139496106781 8500.0   15.0
NE Damien Harris 27.688522885356488 6.993283036412837 7400.0   13.0
LAR Sony Michel 15.05016129032258 6.993283036412837 7100.0   15.0
MIA Jaylen Waddle 12.356719994142127 6.924882303639204 7000.0   14.0
CIN Tee Higgins 15.413378684625236 6.924882303639204 7000.0   13.0
DET Amon-Ra St. Brown 17.455849583709295 6.924882303639204 6900.0   14.0
BUF Devin Singletary 17.962944822531142 6.993283036412837 6000.0   15.0
DAL Dalton Schultz 11.94766672770182 6.928255229099322 5900.0   15.0
DAL Dallas Cowboys 21.154847900109356 6.504761904761905 4200.0   15.0


In [21]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [22]:
output = pd.read_csv('upload.csv')
output

,QB,RB,RB.1,WR,WR.1,WR.2,TE,FLEX,DEF
0,69306-57439:Patrick Mahomes,69306-64576:Damien Harris,69306-53640:Sony Michel,69306-90573:Jaylen Waddle,69306-89981:Tee Higgins,69306-86997:Amon-Ra St. Brown,69306-55042:Dalton Schultz,69306-70603:Devin Singletary,69306-12530:Dallas Cowboys


In [23]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

{'D': 6.504761904761905,
 'QB': 8.736749055620614,
 'RB': 6.993283036412837,
 'TE': 6.928255229099322,
 'WR': 6.924882303639204,
 'DEF': 6.504761904761905,
 'FLEX': 6.959082670026021}

In [24]:
expected_score = sum([m_score.get(h) for h in headers])
expected_score

63.89006184325115

In [25]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [26]:
df[df['First Name'] == 'Jeff']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
56,69306-55552,RB,Jeff,Jeff Wilson Jr.,Wilson Jr.,8.0,6.0,6700,HOU@SF,SF,HOU,NaN,NaN,NaN,NaN,NaN,RB/FLEX,Jeff Wilson Jr.,0.001194


In [27]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [28]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

,FPPG,Nickname,Salary,Salary/FPPG
766,11.933333,Dallas Cowboys,4200,0.002841
913,8.866667,Miami Dolphins,3600,0.002463
909,8.733333,Indianapolis Colts,3900,0.002239
912,7.933333,Denver Broncos,3700,0.002144
910,7.800000,Arizona Cardinals,3900,0.002000
788,7.933333,Kansas City Chiefs,4100,0.001935
283,9.666667,New England Patriots,5000,0.001933
917,6.533333,Cincinnati Bengals,3400,0.001922
457,8.533333,Buffalo Bills,4500,0.001896
304,8.733333,Tampa Bay Buccaneers,4900,0.001782


In [29]:
df[df['Team'] == 'JAC']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG
79,69306-89951,QB,Trevor,Trevor Lawrence,Lawrence,12.473334,15.0,6500,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,QB,Trevor Lawrence,0.001919
114,69306-27986,QB,C.J.,C.J. Beathard,Beathard,0.760000,2.0,6100,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,QB,C.J. Beathard,0.000125
179,69306-38837,QB,Danny,Danny Etling,Etling,0.000000,1.0,6000,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,QB,Danny Etling,0.000000
199,69306-22103,WR,Marvin,Marvin Jones Jr.,Jones Jr.,7.986667,15.0,5700,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Marvin Jones Jr.,0.001401
212,69306-28827,RB,Dare,Dare Ogunbowale,Ogunbowale,2.850000,8.0,5500,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,RB/FLEX,Dare Ogunbowale,0.000518
251,69306-41806,WR,Laquon,Laquon Treadwell,Treadwell,4.430000,10.0,5200,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Laquon Treadwell,0.000852
270,69306-62508,TE,James,James O'Shaughnessy,O'Shaughnessy,5.485715,7.0,5100,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,TE/FLEX,James O'Shaughnessy,0.001076
301,69306-26458,WR,Tavon,Tavon Austin,Austin,3.427273,11.0,5000,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,WR/FLEX,Tavon Austin,0.000685
338,69306-86173,RB,Mekhi,Mekhi Sargent,Sargent,0.300000,3.0,4700,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,RB/FLEX,Mekhi Sargent,0.000064
920,69306-12554,D,Jacksonville,Jacksonville Jaguars,Jaguars,3.133333,15.0,3200,JAC@NE,JAC,NE,NaN,NaN,NaN,NaN,NaN,DEF,Jacksonville Jaguars,0.000979


In [30]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]

HomeTeam       CHI
PointSpread     -6
Name: 1, dtype: object

In [31]:
from pydfs_lineup_optimizer import Site, Sport, get_optimizer

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)
optimizer.load_players_from_csv(ACTIVE_FILE)
for lineup in optimizer.optimize(10):
    print(lineup)


 1. QB      Josh Allen                    QB    BUF            ATL@BUF  24.655         8800.0$   
 2. RB      Cordarrelle Patterson         RB    ATL            ATL@BUF  14.336         6700.0$   
 3. RB      Javonte Williams              RB    DEN            DEN@LAC  11.487         6100.0$   
 4. WR      Cooper Kupp                   WR    LAR            LAR@BAL  21.693         10200.0$  
 5. WR      Cyril Grayson                 WR    TB             TB@NYJ   11.25          4800.0$   
 6. WR      Marquise Brown                WR    BAL            LAR@BAL  12.45          6300.0$   
 7. TE      Dalton Schultz                TE    DAL            ARI@DAL  9.72           5900.0$   
 8. FLEX    Tee Higgins                   WR    CIN            KC@CIN   13.415         7000.0$   
 9. DEF     Dallas Cowboys                D     DAL            ARI@DAL  11.933         4200.0$   

Fantasy Points 130.94
Salary 60000.00

 1. QB      Josh Allen                    QB    BUF            ATL@BUF  24.655